In [3]:
!pip install -qU accelerate peft transformers bitsandbytes

In [ ]:
!gdown 1qcA2mhmK6s_eJteicZZRvQbizJuPlw5L

Downloading...
From: https://drive.google.com/uc?id=1qcA2mhmK6s_eJteicZZRvQbizJuPlw5L
To: /content/adapter.zip
100% 51.0M/51.0M [00:00<00:00, 157MB/s]


In [ ]:
!unzip adapter.zip

Archive:  adapter.zip
   creating: adapter/
  inflating: adapter/adapter_config.json  
  inflating: adapter/adapter_model.safetensors  
 extracting: adapter/added_tokens.json  
  inflating: adapter/README.md       
  inflating: adapter/special_tokens_map.json  
  inflating: adapter/tokenizer.json  
  inflating: adapter/tokenizer.model  
  inflating: adapter/tokenizer_config.json  
  inflating: adapter/training_args.bin  


In [ ]:
import os
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel, PeftConfig

In [ ]:
MODEL_NAME = "/kaggle/input/loraadapter/adapter"
config = PeftConfig.from_pretrained(MODEL_NAME)

model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
#     load_in_8bit=True,
    torch_dtype=torch.float16,
    device_map="auto",
    low_cpu_mem_usage=True,
#     quantization_config=bnb_config,
#     device_map={"": 0}
)
model = PeftModel.from_pretrained(
    model,
    MODEL_NAME,
    torch_dtype=torch.float16
)
model = model.merge_and_unload()

pytorch_model.bin.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training

In [ ]:
model.save_pretrained("/kaggle/working/resume_llm_merged/")
tokenizer.save_pretrained("/kaggle/working/resume_llm_merged/")

In [3]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
model.push_to_hub("resume_llm_merged", use_temp_dir=False)
tokenizer.push_to_hub("resume_llm_merged", use_temp_dir=False)

In [4]:
from huggingface_hub import snapshot_download

snapshot_download(repo_id="Samoed/resume_llm_merged", local_dir='resume_llm_merged')

Fetching 12 files:   0%|          | 0/12 [00:00<?, ?it/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

'/content/resume_llm_merged'

In [5]:
!git clone https://github.com/ggerganov/llama.cpp && cd llama.cpp && make && pip install -r requirements.txt

In [6]:
!python llama.cpp/convert.py resume_llm_merged
!llama.cpp/quantize resume_llm_merged/ggml-model-f16.gguf model_with_lora-q2_k.gguf Q2_K

In [12]:
!huggingface-cli upload Samoed/resume_llm_merged-gguf /content/model_with_lora-q2_k.gguf

In [ ]:
!huggingface-cli upload Samoed/resume_llm_merged-gguf /content/resume_llm_merged/ggml-model-f16.gguf